<a href="https://colab.research.google.com/github/JorgeTabera/DataScienceII.CoderH/blob/main/Proyecto_Spotify_DS_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests
import base64

client_id = "c67b446bd17f48609193652c298217c8"
client_secret = "aa283ef80e594bc8a567390b35e440f9"

auth_str = f"{client_id}:{client_secret}"
b64_auth = base64.b64encode(auth_str.encode()).decode()

headers = {
    "Authorization": f"Basic {b64_auth}",
    "Content-Type": "application/x-www-form-urlencoded"
}

data = {"grant_type": "client_credentials"}

r = requests.post("https://accounts.spotify.com/api/token", headers=headers, data=data)
token = r.json()["access_token"]
print("Token:", token)


playlist_id = "37i9dQZEVXbMMy2roB9myp"
url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
headers = {"Authorization": f"Bearer {TOKEN}"}
params = {"market": "AR", "limit": 50}

r = requests.get(url, headers=headers, params=params)
r.raise_for_status()
data = r.json()



Token: BQB222ugYT4IHiKtOuuJI3wJzfmrOPXiSfjFRP4I7zrCcLAlc8dXvsy0R6Ik4QltF-Vyg9scjYx_W-MU7bry7iiXCLWzeP_0XfTrYlnqs6REck0hiQMU-slIo2YGcTOblFODdRYgm60


NameError: name 'TOKEN' is not defined

In [3]:

playlist_id = "37i9dQZEVXbMMy2roB9myp"
url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
headers = {"Authorization": f"Bearer {TOKEN}"}
params = {"market": "AR", "limit": 50}

r = requests.get(url, headers=headers, params=params)
r.raise_for_status()
data = r.json()


NameError: name 'TOKEN' is not defined